In [49]:
with open("21.in") as f:
    d = f.read()
    lines = d.split("\n")

In [44]:
#     +---+---+
#     | ^ | A |
# +---+---+---+
# | < | v | > |
# +---+---+---+

dir_pad = [
    ["", "^","A"],
    ["<","v",">"]
]

# +---+---+---+
# | 7 | 8 | 9 |
# +---+---+---+
# | 4 | 5 | 6 |
# +---+---+---+
# | 1 | 2 | 3 |
# +---+---+---+
#     | 0 | A |
#     +---+---+
num_pad = [
    ["7","8","9"],
    ["4","5","6"],
    ["1","2","3"],
    ["", "0","A"]
]
'<A<^A^^^AA'

num_pad_shortest = {}

chain = [dir_pad,dir_pad,num_pad]
#chain = [num_pad]

In [28]:
char_to_dir = {"^" : (-1,0), "<" : (0,-1), "v" : (1,0), ">" : (0,1)}
def next_state(state,instruction,chain):
    if len(state) == 0:
        return tuple(),instruction
    
    i,j = state[0]
    pad = chain[0]
    
    if instruction == "A":
        next_instruction = pad[i][j]
        state_and_output = next_state(state[1:],next_instruction,chain[1:])
        if state_and_output is None:
            return None
        ns,o = state_and_output
        return ((i,j),) + ns,o
    
    elif instruction in "^<v>":
        di,dj = char_to_dir[instruction]
        ni = i + di
        nj = j + dj
        
        if ni < 0 or ni >= len(pad) or nj < 0 or nj >= len(pad[0]) or pad[ni][nj] == "":
            return None
        
        return ((ni,nj),) + state[1:], None
    else:
        raise Exception("Not supposed to happen")
        

In [41]:
def sssp(state):
    frontier = {state : 0}
    known = {}

    while len(frontier) > 0:
        smallest_f = None
        smallest_d = None
        #Get smallest frontier thing
        for f,d in frontier.items():
            if smallest_d is None or d < smallest_d:
                smallest_d = d
                smallest_f = f

        #Pop it out of frontier
        frontier.pop(smallest_f)
        known[smallest_f] = smallest_d

        #Update neighbors
        for move in "^<v>A":
            state_and_output = next_state(smallest_f,move,chain)
            if state_and_output is None:
                continue
            s,o = state_and_output
            if s in known:
                continue
            if s in frontier:
                frontier[s] = min(frontier[s],smallest_d+1)
            else:
                frontier[s] = smallest_d + 1
                
    return known

In [45]:
shortest_lengths = {}

for i,row in enumerate(num_pad):
    for j,c in enumerate(row):
        if c == "":
            continue
        state = ((0,2),(0,2),(i,j))
        #state = ((i,j),)
        shortest_lengths[(i,j)] = sssp(state)

In [46]:
def find(button):
    for i,row in enumerate(num_pad):
            for j,c in enumerate(row):
                if c == button:
                    return i,j

def sp(s):
    state = (3,2)
    d = 0
    for c in s:
        new_state = ((0,2),(0,2),find(c))
        #new_state = (find(c),)
        d += shortest_lengths[state][new_state] + 1
        state = new_state[-1]
    return d

In [47]:
sp("029A")

68

In [25]:
len("<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A")

68

In [26]:
sp("0")

19

In [50]:
t = 0
for l in lines:
    t += sp(l)*int(l[:-1])
t

184180

In [51]:
5**25

298023223876953125

# Part 2

## Greedy but too slow

In [60]:
def find2(pad,button):
    for i,row in enumerate(pad):
        for j,c in enumerate(row):
            if c == button:
                return i,j
    print("Not found")

In [91]:
def sign(n):
    return 1 if n > 0 else -1

def what_i_should_push(chain,desired_output):
    if len(chain) == 0: #There are no intervening boards, so I just need to push the buttons
        return desired_output
    
    pad = chain[-1]
    i,j = find2(pad,"A")
    what_this_pad_needs = ""
    for b in desired_output:
        ni,nj = find2(pad,b)
        
        if j < nj:
            hor = ">"
        else:
            hor = "<"
        hors = ""
        hsign = sign(nj - j)
        hfirst = True
        for jj in range(abs(nj - j)):
            hors += hor
            if pad[i][(jj+1)*hsign + j] == "":
                hfirst = False
        
        if i < ni:
            vert = "v"
        else:
            vert = "^"
            
        verts = ""
        for ii in range(abs(ni - i)):
            verts += vert
                    
        if hfirst:
            what_this_pad_needs += hors
            what_this_pad_needs += verts
        else:
            what_this_pad_needs += verts
            what_this_pad_needs += hors

        what_this_pad_needs += "A"
        
        i,j = ni,nj
            
    return what_i_should_push(chain[:-1],what_this_pad_needs)

In [ ]:
num_pad = [
    ["7","8","9"],
    ["4","5","6"],
    ["1","2","3"],
    ["", "0","A"]
]

In [87]:
what_i_should_push(chain,lines[0])

(3, 2) (2, 0)
(2, 0) (2, 1)
(2, 1) (0, 2)
(0, 2) (3, 2)
(0, 2) (0, 1)
(0, 1) (1, 0)
(1, 0) (1, 0)
(1, 0) (0, 2)
(0, 2) (1, 2)
(1, 2) (0, 1)
(0, 1) (0, 2)
(0, 2) (1, 2)
(1, 2) (0, 1)
(0, 1) (0, 2)
(0, 2) (1, 1)
(1, 1) (0, 2)
(0, 2) (1, 0)
(1, 0) (0, 1)
(0, 1) (0, 2)
(0, 2) (1, 1)
(1, 1) (1, 0)
(1, 0) (0, 2)
(0, 2) (0, 1)
(0, 1) (0, 2)
(0, 2) (1, 2)
(1, 2) (1, 2)
(1, 2) (0, 1)
(0, 1) (0, 2)
(0, 2) (1, 1)
(1, 1) (0, 2)
(0, 2) (1, 0)
(1, 0) (0, 1)
(0, 1) (0, 2)
(0, 2) (1, 2)
(1, 2) (0, 1)
(0, 1) (0, 2)
(0, 2) (1, 1)
(1, 1) (0, 2)
(0, 2) (1, 0)
(1, 0) (0, 1)
(0, 1) (0, 2)
(0, 2) (1, 2)
(1, 2) (0, 1)
(0, 1) (0, 2)
(0, 2) (1, 0)
(1, 0) (1, 1)
(1, 1) (0, 2)
(0, 2) (1, 2)
(1, 2) (0, 1)
(0, 1) (0, 2)


'v<<A>^A>^A<vA<^A>>^A<^A>^AvA^A<^A>^A<vA>^Av<<A>^A>^AvA<^A>^A<vA>^Av<<A>^A>^AvA<^A>^Av<<A>^A>^AvA<^A>^A'

In [92]:
what_i_should_push(chain[-1:],"029A")

'<A^A>^^AvvvA'

In [93]:
t = 0
for l in lines:
    t += len(what_i_should_push(chain,l))*int(l[:-1])
t

186508

In [71]:
t = 0
for l in lines:
    t += len(what_i_should_push(chain,l))*int(l[:-1])
t

185466

In [63]:
%debug

> c:\users\josh\appdata\local\temp\ipykernel_30736\4062241264.py(9)what_i_should_push()

ipdb> pad
[['', '^', 'A'], ['<', 'v', '>']]
ipdb> print(b)
1
ipdb> chain
[[['', '^', 'A'], ['<', 'v', '>']], [['', '^', 'A'], ['<', 'v', '>']]]
ipdb> exit


In [94]:
thing = {"a" : 1, "b" : 2}


1

In [116]:
char_to_dir = {"^" : (-1,0), "<" : (0,-1), "v" : (1,0), ">" : (0,1)}


def ss_cost(source,arrow_cost,pad):
    state = ("A",source)
    
    frontier = {state : 0}
    known = {}
    
    while len(frontier) > 0:
        
        d = min(frontier.values())
        f = [k for k,v in frontier.items() if v == d][0]
        
        frontier.pop(f)
        known[f] = d
        
        for b in "<v>^A":
            if b == "A":
                new_state = ("A",f[1])
                new_d = d + arrow_cost[f[0]][b]
            else:
                i,j = find2(pad,f[1])
                di,dj = char_to_dir[b]
                ni = i + di
                nj = j + dj
                if ni < 0 or ni >= len(pad) or nj < 0 or nj >= len(pad[0]) or pad[ni][nj] == "":
                    continue
                new_state = (b,pad[ni][nj])
                new_d = d + arrow_cost[f[0]][b] + 1

            
            if new_state in known:
                continue
                
            if new_state in frontier:
                frontier[new_state] = min(frontier[new_state],new_d)
            else:
                frontier[new_state] = new_d
                
    to_return = {}
    for state in known:
        if state[0] == "A":
            to_return[state[1]] = known[state]
            
    return to_return
            
        

def get_cost(arrow_cost,pad):
    to_return = {}
    for row in pad:
        for key in row:
            if key != "":
                to_return[key] = ss_cost(key,arrow_cost,pad)
    return to_return
        
    

In [99]:
buttons = "<v>^A"
zeros = {key : 0 for key in buttons}
typing_cost = {key : zeros for key in buttons}

In [117]:
get_cost(typing_cost,)

{'7': {'7': 0,
  '4': 1,
  '8': 1,
  '1': 2,
  '5': 2,
  '9': 2,
  '2': 3,
  '6': 3,
  '0': 4,
  '3': 4,
  'A': 5},
 '8': {'8': 0,
  '7': 1,
  '5': 1,
  '9': 1,
  '4': 2,
  '2': 2,
  '6': 2,
  '1': 3,
  '0': 3,
  '3': 3,
  'A': 4},
 '9': {'9': 0,
  '8': 1,
  '6': 1,
  '7': 2,
  '5': 2,
  '3': 2,
  '4': 3,
  '2': 3,
  'A': 3,
  '1': 4,
  '0': 4},
 '4': {'4': 0,
  '1': 1,
  '5': 1,
  '7': 1,
  '2': 2,
  '6': 2,
  '8': 2,
  '0': 3,
  '3': 3,
  '9': 3,
  'A': 4},
 '5': {'5': 0,
  '4': 1,
  '2': 1,
  '6': 1,
  '8': 1,
  '1': 2,
  '7': 2,
  '0': 2,
  '3': 2,
  '9': 2,
  'A': 3},
 '6': {'6': 0,
  '5': 1,
  '3': 1,
  '9': 1,
  '4': 2,
  '2': 2,
  '8': 2,
  'A': 2,
  '1': 3,
  '7': 3,
  '0': 3},
 '1': {'1': 0,
  '2': 1,
  '4': 1,
  '0': 2,
  '3': 2,
  '5': 2,
  '7': 2,
  'A': 3,
  '6': 3,
  '8': 3,
  '9': 4},
 '2': {'2': 0,
  '1': 1,
  '0': 1,
  '3': 1,
  '5': 1,
  '4': 2,
  'A': 2,
  '6': 2,
  '8': 2,
  '7': 3,
  '9': 3},
 '3': {'3': 0,
  '2': 1,
  'A': 1,
  '6': 1,
  '1': 2,
  '0': 2,
  '5': 

In [ ]:
#     +---+---+
#     | ^ | A |
# +---+---+---+
# | < | v | > |
# +---+---+---+


In [122]:
part_1_cost = get_cost(get_cost(get_cost(typing_cost,dir_pad),dir_pad),num_pad)

In [125]:
button = "A"
t = 0
for c in "029A":
    t += part_1_cost[button][c] + 1
    button = c
t

68

In [126]:
part_2_cost = get_cost(typing_cost,dir_pad) #One robot
for i in range(24):
    part_2_cost = get_cost(part_2_cost,dir_pad)
part_2_cost = get_cost(part_2_cost,num_pad)

In [127]:
def s_cost(s,cost_function):
    button = "A"
    t = 0
    for c in s:
        t += cost_function[button][c] + 1
        button = c
    return t

In [129]:
s_cost("029A",part_2_cost)

82050061710

In [130]:
t = 0
for l in lines:
    t += s_cost(l,part_2_cost)*int(l[:-1])
t

231309103124520